# Bridging CT segmentation and nodule candidate classification
## NoduleAnalysis App
### NoduleAnalysisApp.main()
The next step is to convert the segmentation output into sample tuples. Grouping the data involves finding the dashed outline around what;s flagged by the model, and then taking the segmentation output, which is the voxels flagged by the segmentation model, and coordfinate the center of each lump of flagged voxels as index, row and column.  
#### LunaDataset (val stride=10, and isvallset = true)
First time the LunaDataset is called by running the nodule_analysis.py script, there is no values inside the candidateInfo_list, hence why we extract the values while using cache. 
1. If it's the validation set, then extract every Xth (val_stride) value from the series set, otherwise delete the validation entries and that's your training set. The splitting is at the CT/series_uid level. __getitem__ atribute.  When accessing an it gives batch_size * 48x48 matrices. 
2. For nodule candidatw generation, we iterate over all the series_set to extract their corresponding CT. Each object includes the 121x512x512 CT scan. 
3. <strong>segmentCt</strong> <br>
This line of code runs the segmentation model on the CT scan.
    - <strong>a.</strong> seg_dl<br> 
    Every CT scan has 121 slices the index direction. When accessing an index of the Segmentation dataset, we are given a 7x512x512 CT scan, a posiive mask of dimensions 1x512x512, the series_uid and the slice index.
    -  <strong>b.</strong><br> The segmentation model takes batch_size x 7 context slices x 512 x 512 as input, which means in terms of width and length covers the entire area of the CT scan. The output is batch_size x 1 x 512 x 512. The prediction is a boolean mask
    Note that during training, the last two dimensions of the input if 64 by 64, representing just voxels. 
    -  <strong>c.</strong><br> Prior to this, we've been able to create the boolean mask for all channels accross the CT scan. Here we apply binary erosion which is a mathematical morhology structuring element for shrinking the shapes in an image. 

4. <strong>groupSegmentationOutput</strong> <br>

- a. The label function will take nonzero pixels and mark them as belonging to the same group. The second part of the output is the number of objects found.
- b. Gets the center mass coordinates for each group in the IRC coordinates. This function is intented for center mass calculation, which is why we offset the array by adding 1001 to all pixel density values. 
- c. Here we build our candidate info tuple and append it to the list of detections. 
5. <strong>Nodule Classification</strong> <br>
The classification model resides in the classifications directory. The LunaModel is trained on the LunaData. The LunaData are basically 32x48x48 data that are being classified 0(non-nodule) or 1 (nodule). A sample that is not a nodule has [1, 0] as a label. 
- a. Column 1 of the probability predictions is the predicted probability that should be kept. 
- b. We are zipping the IRC center coordinates, nodule probability, malignancy probability, and also we get the center coordinates in XYZ representation. 
6. If we have the ground truth data which resides in the candidateInfo_dict, compute and pring the confusion matrix and add the results to the total.
- a. The Complete Miss column is basicaly Nodules that were completely missed, wether it they were Benign or Malignant. Anythings that makes it through the predicted nodules, are the ones that survived the nodule classifier. Any non-nodules under that column are basically False Positive (falsely classified as nodules). The number of Non-nodules that were filtered out were the ones that didn't even make it to the malignancy model, as they correctly had been classified as non-nodules.  